In [30]:
# Initial imports
import pandas as pd
import hvplot.pandas
from path import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

In [31]:
crypto_df = pd.read_csv("crypto_data.csv", index_col='Unnamed: 0')

In [32]:
# Keep all the cryptocurrencies that are being traded.
crypto_df = crypto_df.loc[crypto_df['IsTrading'] == True]
crypto_df.drop(columns=['IsTrading'], inplace=True)

In [33]:
# Drop rows with null values
crypto_df.dropna(inplace=True)
crypto_df

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
...,...,...,...,...,...
ZEPH,ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000
GAP,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,PoW,7.296538e+06,21000000


In [34]:
# Keep the rows where coins are mined.
crypto_df = crypto_df.loc[crypto_df['TotalCoinsMined'] > 0]

In [35]:
# Create a new DataFrame that holds only the cryptocurrencies names.
crypto_names = crypto_df[['CoinName']]
crypto_names

,CoinName
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum
...,...
ZEPH,ZEPHYR
GAP,Gapcoin
BDX,Beldex
ZEN,Horizen


In [36]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.
crypto_df.drop(columns=['CoinName'], inplace=True)
crypto_df

/Users/travisdharry/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/pandas/core/frame.py:4315: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0
...,...,...,...,...
ZEPH,SHA-256,DPoS,2.000000e+09,2000000000
GAP,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,CryptoNight,PoW,9.802226e+08,1400222610
ZEN,Equihash,PoW,7.296538e+06,21000000


In [37]:
# Use get_dummies() to create variables for text features.
df_binary = pd.get_dummies(crypto_df)
df_binary

,TotalCoinsMined,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,Algorithm_Cloverhash,...,TotalCoinSupply_91388946,TotalCoinSupply_92000000000,TotalCoinSupply_9354000,TotalCoinSupply_9507271,TotalCoinSupply_9736000,TotalCoinSupply_98000000,TotalCoinSupply_98100000000,TotalCoinSupply_990000000000,TotalCoinSupply_999481516,TotalCoinSupply_9999999
42,4.199995e+01,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
404,1.055185e+09,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZEPH,2.000000e+09,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
GAP,1.493105e+07,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
BDX,9.802226e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ZEN,7.296538e+06,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [38]:
# Standardize the data with StandardScaler().
crypto_scaled = StandardScaler().fit_transform(df_binary)
print(crypto_scaled[0:5])

[[-0.11710817 -0.0433963  -0.0433963  ... -0.0433963  -0.0433963
  -0.0433963 ]
 [-0.09396955 -0.0433963  -0.0433963  ... -0.0433963  -0.0433963
  -0.0433963 ]
 [ 0.52494561 -0.0433963  -0.0433963  ... -0.0433963  -0.0433963
  -0.0433963 ]
 [-0.11671506 -0.0433963  -0.0433963  ... -0.0433963  -0.0433963
  -0.0433963 ]
 [-0.11474682 -0.0433963  -0.0433963  ... -0.0433963  -0.0433963
  -0.0433963 ]]


In [39]:
# Using PCA to reduce dimension to three principal components.
pca = PCA(n_components=3)
crypto_pca = pca.fit_transform(crypto_scaled)
crypto_pca

array([[-0.18622046, -0.06663607,  0.04258698],
       [-0.18337108, -0.06555462,  0.04511611],
       [ 0.4423876 , -0.10819713,  0.17410126],
       ...,
       [-0.53483357, -0.11092184, -0.09485162],
       [-0.60950117, -0.06614279, -0.00728508],
       [ 0.05473412, -0.0768299 ,  0.07301885]])

In [40]:
# Create a DataFrame with the three principal components.
df_crypto_pca = pd.DataFrame(data=crypto_pca, columns=['PC 1', 'PC 2', 'PC 3'], index=df_binary.index)
df_crypto_pca.head()

,PC 1,PC 2,PC 3
42,-0.186220,-0.066636,0.042587
404,-0.183371,-0.065555,0.045116
1337,0.442388,-0.108197,0.174101
BTC,-0.495635,-0.082370,-0.030580
ETH,-0.320003,-0.165318,-0.110185


In [41]:
# Create an elbow curve to find the best value for K.
# Find the best value for K
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of K values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(df_crypto_pca)
    inertia.append(km.inertia_)

# Create the elbow curve
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow

,k,inertia
0,1,4852.894041
1,2,3281.564230
2,3,1951.354023
3,4,606.918378
4,5,349.441175
5,6,142.163993
6,7,92.371971
7,8,68.044379
8,9,54.352193
9,10,41.323345


In [42]:
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")

:Curve   [k]   (inertia)

In [43]:
# Initialize the K-Means model.
# Initialize the K-means model
model = KMeans(n_clusters=4, random_state=0)

# Fit the model
model.fit(df_crypto_pca)

# Predict clusters
predictions = model.predict(df_crypto_pca)

# Add the predicted class columns
df_crypto_pca["class"] = model.labels_
df_crypto_pca.head()

,PC 1,PC 2,PC 3,class
42,-0.186220,-0.066636,0.042587,0
404,-0.183371,-0.065555,0.045116,0
1337,0.442388,-0.108197,0.174101,0
BTC,-0.495635,-0.082370,-0.030580,0
ETH,-0.320003,-0.165318,-0.110185,0


In [44]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.
clustered_df = pd.concat([crypto_df, df_crypto_pca], axis=1)
clustered_df.drop(['class'], axis=1, inplace=True)

#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
clustered_df = pd.concat([clustered_df, crypto_names], axis=1)

#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions.
clustered_df = pd.concat([clustered_df, df_crypto_pca['class']], axis=1)

# Print the shape of the clustered_df
print(clustered_df.shape)
clustered_df.head(10)

(532, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,CoinName,class
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.186220,-0.066636,0.042587,42 Coin,0
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.183371,-0.065555,0.045116,404Coin,0
1337,X13,PoW/PoS,2.927942e+10,314159265359,0.442388,-0.108197,0.174101,EliteCoin,0
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.495635,-0.082370,-0.030580,Bitcoin,0
ETH,Ethash,PoW,1.076842e+08,0,-0.320003,-0.165318,-0.110185,Ethereum,0
LTC,Scrypt,PoW,6.303924e+07,84000000,-0.540325,-0.128438,-0.003636,Litecoin,0
DASH,X11,PoW/PoS,9.031294e+06,22000000,-0.208751,-0.025932,-0.037260,Dash,0
XMR,CryptoNight-V7,PoW,1.720114e+07,0,-0.212502,-0.145933,-0.068579,Monero,0
ETC,Ethash,PoW,1.133597e+08,210000000,-0.502634,-0.126194,-0.121424,Ethereum Classic,0
ZEC,Equihash,PoW,7.383056e+06,21000000,-0.609500,-0.066143,-0.007285,ZCash,0


In [45]:
# Creating a 3D-Scatter with the PCA data and the clusters
# YOUR CODE HERE
fig = px.scatter_3d(
    clustered_df,
    x="PC 1",
    y="PC 2",
    z="PC 3",
    color="class",
    symbol="class",
    width=800,
    hover_name = "CoinName"
)
fig.update_layout(legend=dict(x=0, y=1))
fig.show()

In [46]:
# Create a table with tradable cryptocurrencies.
clustered_df.hvplot.table(columns=['CoinName', 'Algorithm', 'ProofType', 'TotalCoinSupply', 'TotalCoinsMined', 'class'], sortable=True, selectable=True)

:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,class]

In [52]:
# Print the total number of tradable cryptocurrencies.
tradable_count = len(clustered_df)
print(tradable_count)

532


In [53]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
clustered_scaled = StandardScaler().fit_transform(clustered_df[['TotalCoinSupply', 'TotalCoinsMined']])
print(clustered_scaled[0:5])

[[-0.1528703  -0.11710817]
 [-0.145009   -0.09396955]
 [ 4.48942416  0.52494561]
 [-0.15255998 -0.11671506]
 [-0.1528703  -0.11474682]]


In [60]:
# Create a new DataFrame that has the scaled data with the clustered_df DataFrame index.
# Add the "CoinName" column from the clustered_df DataFrame to the new DataFrame.
# Add the "Class" column from the clustered_df DataFrame to the new DataFrame. 
plot_df = pd.DataFrame(data=clustered_scaled, columns=['TotalCoinSupply', 'TotalCoinsMined'], index=clustered_df.index)
plot_df['CoinName'] = clustered_df[['CoinName']]
plot_df['Class'] = clustered_df[['class']]
plot_df.head(10)

,TotalCoinSupply,TotalCoinsMined,CoinName,Class
42,-0.152870,-0.117108,42 Coin,0
404,-0.145009,-0.093970,404Coin,0
1337,4.489424,0.524946,EliteCoin,0
BTC,-0.152560,-0.116715,Bitcoin,0
ETH,-0.152870,-0.114747,Ethereum,0
LTC,-0.151629,-0.115726,Litecoin,0
DASH,-0.152545,-0.116910,Dash,0
XMR,-0.152870,-0.116731,Monero,0
ETC,-0.149767,-0.114622,Ethereum Classic,0
ZEC,-0.152560,-0.116946,ZCash,0


In [64]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
plot_df.hvplot.scatter(
    x="TotalCoinsMined",
    y="TotalCoinSupply",
    hover_cols=["CoinName"],
    by="Class",
)


:NdOverlay   [Class]
   :Scatter   [TotalCoinsMined]   (TotalCoinSupply,CoinName)